# LongT5 pretraining with color-responsive dataset

Input is too long for T5's 512 max input sequence length.

In [1]:
import shutil
!pip install -U git+https://github.com/huggingface/transformers.git@13493215abceafc1653af88b045120014fb4c1fc
!pip install -U git+https://github.com/huggingface/accelerate.git@8ade23cc6aec7c3bd3d80fef6378cafaade75bbe
!pip install -U git+https://github.com/huggingface/datasets.git@01f91bae037c98f2e05456287bab21470adb8f07
!pip install -U git+https://github.com/huggingface/evaluate.git@55f1bc6e072b05c2d9db1589a07e20f38902b1ec
!pip install -U git+https://github.com/huggingface/safetensors.git@f5839b6aee407652aa3078d91206b618dd84e3c2

!pip install sacrebleu
!pip install pytest-playwright
!playwright install
!pip install matplotlib
!pip install pillow
!pip install torchvision
!pip install lpips

!playwright install-deps  

!pip install -U numpy
!pip install tensorboard

  Cloning https://github.com/huggingface/transformers.git (to revision 13493215abceafc1653af88b045120014fb4c1fc) to /tmp/pip-req-build-s5gi571r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-s5gi571r
  Running command git rev-parse -q --verify 'sha^13493215abceafc1653af88b045120014fb4c1fc'
  Running command git fetch -q https://github.com/huggingface/transformers.git 13493215abceafc1653af88b045120014fb4c1fc
  Running command git checkout -q 13493215abceafc1653af88b045120014fb4c1fc
  Resolved https://github.com/huggingface/transformers.git to commit 13493215abceafc1653af88b045120014fb4c1fc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 7.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!apt install zip -y
!rm -rf data-rb-color-responsive
!mkdir -p data-rb-color-responsive
!wget "https://www.dropbox.com/scl/fi/ei7ere3ecrr58ovgbps3i/color-responsive.zip.zip?rlkey=7dnmece3kixgk7s7khds27xu5&st=hufvmpnc&dl=1" -O model.zip
!unzip model.zip -d data-rb-color-responsive

!mkdir -p data-rb-validate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  unzip
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 43 not upgraded.
Need to get 350 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2 [175 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2 [176 kB]
Fetched 350 kB in 1s (348 kB/s)m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 34449 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 11%] [######................................

In [3]:
from datasets import load_from_disk, DatasetDict

validation_size = 4
test_size = 100

dataset = load_from_disk('data-rb-color-responsive')

train_test_split = dataset.train_test_split(test_size=test_size/len(dataset))
train_valid_split = train_test_split['train'].train_test_split(test_size=validation_size/len(dataset))

dataset = DatasetDict({
    "train": train_valid_split["train"],
    "valid": train_valid_split["test"],
    "test": train_test_split["test"],
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['svg', 'html'],
        num_rows: 100408
    })
    valid: Dataset({
        features: ['svg', 'html'],
        num_rows: 4
    })
    test: Dataset({
        features: ['svg', 'html'],
        num_rows: 100
    })
})


In [4]:
print(dataset['train'][0])

{'svg': 'Mobile: \n\n<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="393" height="852" x="0" y="0" fill="rgb(143, 35, 190)"/></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>\n\nDesktop:\n\n<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="1440" height="900" viewBox="0 0 1440 900"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" 

In [5]:
from transformers import AutoTokenizer
from tokenizers.processors import TemplateProcessing

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
new_words = ['<', '>', '{', '}', '<body>', '<div>', '</div>', '<style>', '</style>', '</body>']
tokenizer.add_tokens(new_words)

config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

10

In [6]:
tokenized_output = tokenizer.encode(dataset['train'][0]['svg'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized SVG contains unknown tokens.")
else:
    print("No unknown tokens found in the SVG. Length: " + str(len(tokenized_output)))

tokenized_output = tokenizer.encode(dataset['train'][0]['html'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized HTML contains unknown tokens.")
else:
    print("No unknown tokens found in the HTML. Length: " + str(len(tokenized_output)))

No unknown tokens found in the SVG. Length: 691
No unknown tokens found in the HTML. Length: 48


In [7]:
print(tokenizer.decode(tokenizer.encode(dataset['train'][0]['html'], add_special_tokens=False)))

<body></body><style> body{ background-color: #8f23be; margin: 0;} @media (min-width: 768px){ body{ background-color: #246555;}}</style>


In [8]:
MAX_SEQUENCE_LENGTH = 1024

def get_token_lengths(examples):
    inputs = tokenizer(
        examples['svg'],
        truncation=False,  # Don't truncate yet
        padding=False,     # Don't pad yet
        return_length=True,
    )
    labels = tokenizer(
        examples['html'],
        truncation=False,
        padding=False,
        return_length=True,
    )

    inputs["input_length"] = inputs["length"]
    inputs["labels"] = labels["input_ids"]
    inputs["label_length"] = labels["length"]

    inputs.pop("length", None)

    return inputs

tokenized_data = dataset.map(get_token_lengths, batched=True)

def filter_function(example):
    return example['input_length'] <= MAX_SEQUENCE_LENGTH and example['label_length'] <= MAX_SEQUENCE_LENGTH

filtered_data = tokenized_data.filter(filter_function)

def tokenize_examples(examples):
    inputs = tokenizer(
        examples['svg'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )
    labels = tokenizer(
        examples['html'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )

    labels = labels['input_ids']
    labels[labels == tokenizer.pad_token_id] = -100
    inputs["labels"] = labels
    

    return inputs

filtered_data = filtered_data.map(get_token_lengths, batched=True, remove_columns=dataset["train"].column_names)

print(filtered_data)

Map:   0%|          | 0/100408 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100408 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100408 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 100408
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 4
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 100
    })
})


In [9]:
import numpy as np
from utils.similarity import calculate_metrics
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torch
from types import SimpleNamespace
import shutil

VIEWPORT_SIZES = {
    'DESKTOP': {'width': 1440, 'height': 900},
    'TABLET': {'width': 834, 'height': 1210},
    'MOBILE': {'width': 393, 'height': 852},
}

VIEWPORTS = ['MOBILE', 'DESKTOP']

def add_image_to_tensorboard(name, step, img_path):
    image = Image.open(img_path)
    image = image.convert('RGB')
    image_array = np.array(image)
    image_tensor = torch.from_numpy(image_array)
    image_tensor = image_tensor.permute(2, 0, 1)
    image_tensor = image_tensor.float() / 255.0
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_image(name, image_tensor, step)
    
def add_text_to_tensorboard(name, step, text):
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_text(name, text, step)

def postprocess_text(preds, labels):
    preds = [pred.strip().replace('<unk>', '') for pred in preds]
    labels = [[label.strip().replace('<unk>', '')] for label in labels]

    return preds, labels

def create_compute_metrics(trainer, save_best_model_dir = None):
    best_perceptual_loss = None

    def compute_metrics(eval_predictions, to_tensorboard = True):
        nonlocal best_perceptual_loss

        predictions, labels = eval_predictions
        
        if isinstance(predictions, tuple):
            predictions = predictions[0]

        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_predictions, decoded_labels = postprocess_text(decoded_predictions, decoded_labels)
    
        similarity_scores = []
        perceptual_losses = []
        index = 1
        for prediction, label in zip(decoded_predictions, decoded_labels):
            if to_tensorboard:
                add_text_to_tensorboard(f'valid_{index}_label_text', trainer.state.global_step, label[0])
                add_text_to_tensorboard(f'valid_{index}_prediction_text', trainer.state.global_step, prediction)

            for viewport in VIEWPORTS:
            
                metrics = calculate_metrics(prediction, label[0],
                                        VIEWPORT_SIZES[viewport]['width'], VIEWPORT_SIZES[viewport]['height'])

                if metrics is not None:
                    similarity_scores.append(metrics['similarity'])
                    perceptual_losses.append(metrics['perceptual_loss'])

                    if to_tensorboard:
                        add_image_to_tensorboard(f'valid_{index}_expectation_' + viewport.lower(), trainer.state.global_step, metrics['expected_screenshot_path'])
                        add_image_to_tensorboard(f'valid_{index}_prediction_' + viewport.lower(), trainer.state.global_step, metrics['predicted_screenshot_path'])
            
            index += 1


        result = {
            'similarity': float(np.mean(similarity_scores)),
            'perceptual_loss': float(np.mean(perceptual_losses)),
        }

        if save_best_model_dir is not None:
            if best_perceptual_loss is None or result['perceptual_loss'] < best_perceptual_loss:
                if os.path.exists(save_best_model_dir):
                    shutil.rmtree(save_best_model_dir)
                trainer.save_model(save_best_model_dir)
                tokenizer.save_pretrained(save_best_model_dir)
                with open(os.path.join(save_best_model_dir, 'best_results.txt'), 'w') as f:
                    f.write(str(result) + '\nStep: ' + str(trainer.state.global_step))

                best_perceptual_loss = result['perceptual_loss']
    
        result = {k: round(v, 4) for k, v in result.items()}
        return result
    
    return compute_metrics

def eval_outside_of_training(model, dataset, to_tensorboard = False, step = None):
    def pad_to_matrix(seqs, pad_value=0):
        max_len = max(len(s) for s in seqs)
        matrix  = np.full((len(seqs), max_len), pad_value, dtype=np.int64)
        for i, s in enumerate(seqs):
            matrix[i, :len(s)] = s                 # left-align, pad on the right
        return matrix
    
    model.eval()
    model.to('cuda:0')

    predictions = []
    labels = []
    for row in dataset:
        inputs = tokenizer(row['svg'], return_tensors="pt").input_ids
        labels.append(tokenizer(row['html'], return_tensors="pt").input_ids)
        inputs = inputs.to('cuda:0')
        outputs = model.generate(inputs)
        predictions.append(outputs[0])

    mock_trainer = SimpleNamespace(state=SimpleNamespace(global_step=step))
    compute_metrics = create_compute_metrics(mock_trainer)
    return compute_metrics((
            pad_to_matrix([t.cpu().numpy() for t in predictions]), 
            pad_to_matrix([t.squeeze(0).cpu().numpy() for t in labels])
        ), to_tensorboard=to_tensorboard)



In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google/long-t5-tglobal-base")

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoConfig

config = AutoConfig.from_pretrained(
    "google/long-t5-tglobal-base",
    vocab_size=len(tokenizer),
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id,
    decoder_start_token_id = tokenizer.pad_token_id,
)

model = AutoModelForSeq2SeqLM.from_config(config)
model.resize_token_embeddings(len(tokenizer))

Embedding(32109, 768)

In [12]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_model_config(model.config)
generation_config._from_model_config = False
generation_config.max_new_tokens = 1024

generation_config

GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "max_new_tokens": 1024,
  "pad_token_id": 0
}

In [13]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_output_dir = f"model-{timestamp}"
log_dir = f'{model_output_dir}/runs'

In [14]:
results = eval_outside_of_training(model, dataset["valid"], True, 0)
writer = SummaryWriter(log_dir=log_dir)   # or reuse the one you have
for k, v in results.items():
    writer.add_scalar(f"eval/{k}", v, 0)

In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import os

training_args = Seq2SeqTrainingArguments(
    output_dir=model_output_dir,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # gradient_accumulation_steps=16,
    # gradient_checkpointing=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    logging_steps=25,
    logging_dir=log_dir,
    predict_with_generate=True,
    # load_best_model_at_end=True,
    bf16=True,
    generation_config=generation_config,
    report_to="tensorboard",
    save_safetensors=False # Fixing missing encoder.embed_tokens.weight and decoder.embed_tokens.weight due to a bug.
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=filtered_data["train"],
    eval_dataset=filtered_data["valid"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,
)

trainer.compute_metrics = create_compute_metrics(trainer, os.path.join(model_output_dir, "best"))

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Similarity,Perceptual Loss,Runtime,Samples Per Second,Steps Per Second
1000,4.176100,1.975760,0.627500,0.558900,20.598100,0.194000,0.049000
2000,3.811200,1.818287,0.757000,0.489600,20.436800,0.196000,0.049000
3000,3.616700,1.655217,0.645200,0.550400,20.411900,0.196000,0.049000
4000,3.434900,1.671751,0.671700,0.562200,20.259500,0.197000,0.049000
5000,3.281100,1.519780,0.632200,0.558700,20.145800,0.199000,0.050000
6000,2.962300,1.857679,0.655200,0.552100,20.169800,0.198000,0.050000
7000,2.566200,1.442396,0.840900,0.461000,20.831400,0.192000,0.048000
8000,2.057000,0.988202,0.728500,0.452900,20.414000,0.196000,0.049000
9000,1.504300,0.438864,0.932400,0.232600,20.418300,0.196000,0.049000
10000,1.183100,0.435434,0.848400,0.263900,20.594400,0.194000,0.049000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=62760, training_loss=0.6069192497648235, metrics={'train_runtime': 13197.1161, 'train_samples_per_second': 76.083, 'train_steps_per_second': 4.756, 'total_flos': 9.322508446442496e+17, 'train_loss': 0.6069192497648235, 'epoch': 10.0})

In [16]:
import os

trainer.save_model(os.path.join(model_output_dir, "final"))
tokenizer.save_pretrained(os.path.join(model_output_dir, "final"))

('model-2025-04-24_13-50-55/final/tokenizer_config.json',
 'model-2025-04-24_13-50-55/final/special_tokens_map.json',
 'model-2025-04-24_13-50-55/final/tokenizer.json')

In [17]:
from transformers import LongT5ForConditionalGeneration

text = dataset['test'][0]['svg']

inputs = tokenizer(text, return_tensors="pt").input_ids
inputs = inputs.to('cuda:0')

model = LongT5ForConditionalGeneration.from_pretrained(os.path.join(model_output_dir, "best"))
model.to('cuda:0')
model.generation_config = generation_config

outputs = model.generate(inputs)
# outputs = model.generate(inputs, max_new_tokens=16_384, do_sample=True, top_k=30, top_p=0.95)

translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(dataset['test'][0]['html'])
print(translation)


<body></body>

<style>


        body {
            background-color: #63816c;
            margin: 0;
        }

        @media (min-width: 768px) {
            body {
                background-color: #14a097;
            }
        }

    

</style>
<body></body><style> body{ background-color: #0081c; margin: 0;} @media (min-width: 768px){ body{ background-color: #143a97;}}</style>


In [18]:
test_results = eval_outside_of_training(model, dataset["test"], False)

print(test_results)

{'similarity': 0.948, 'perceptual_loss': 0.0989}
